In [ ]:
from river.datasets import synth
from clustream import CluStream
from river import drift
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd
from scipy.spatial import distance_matrix

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA
import itertools




In [ ]:
ds = pd.read_csv('./datasets/rbf speed 20L/rbf_speed_20L.csv')
#ds = pd.read_csv('./datasets/rbf create delete 2/rbf_create_delete_2.csv')
df = ds.copy()
y = df.pop('.class')
df.head()

In [ ]:
model = CluStream(n_macro_clusters=3,max_micro_clusters=12,micro_cluster_r_factor=0.005,time_window=10)

In [ ]:
fig ,ax = plt.subplots(2,3,figsize=(10,6),sharex=True, sharey= True)
it = itertools.product((0,1,2),(0,1,2))
colors = {1 : 'blue', 2 : 'red', 3 : 'green' }
i = 0
go = True
dist = []
for x,y in stream.iter_pandas(df,y):
    model = model.learn_one(x)
    i +=1
    if model._initialized:
        pass

    if i % 500 == 0 :
        
        k,m = next(it)
        centroids = model.get_micro_clusters()
        grouped = ds[i:i+250].groupby('.class')
        for key,group in grouped :
            group.plot(ax=ax[k,m],x='X1',y='X2',kind="scatter",label = key,color=colors[key],title=f'from {i} to {i + 250}')
        ax[k,m].scatter(centroids[:,0],centroids[:,1],marker="x",c='black')
    
    if i ==3000 :
        break
    
    
        

   
fig.savefig('window_150_30mc')